In [ ]:
pip install pypdf nltk spacy

In [ ]:
!python -m spacy download en_core_web_sm 

In [ ]:
!python -m spacy download en_core_web_trf

In [33]:
# importing required modules 
from pypdf import PdfReader 

# creating a pdf reader object 
reader = PdfReader('../embed_docs/Numenera Discovery.pdf') 

# printing number of pages in pdf file 
print(len(reader.pages)) 


418


In [ ]:

# getting a specific page from the pdf file 
page = reader.pages[52] 

# extracting text from page 
text = page.extract_text() 
print(text) 

In [35]:
from pypdf import PdfReader 
import nltk

file = '../embed_docs/Numenera Discovery.pdf'

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = " ".join(page.extract_text() for page in pdf.pages)
    return text

# Extract text from the PDF and split it into sentences
text = extract_text_from_pdf(file)
print(len(text))

1597961


In [ ]:
sample = text[31015:33037]
print(sample)

In [4]:
# Splitting Text into Sentences
def split_text_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

sentences = split_text_into_sentences(text)
print(len(sentences))

16731


This gets us text, sure, but what do we do with it?

In [21]:
import ollama
from pprint import pprint
model_list = ollama.list()
for model in model_list['models']:
    model_name = model['model']  # Access the 'model' attribute directly from each model
    pprint(model_name)

'mistral:7b'
'mistral:latest'
'mxbai-embed-large:v1'
'phi:2.7b'


Let's try a summarization with the doc object

In [ ]:
# importing required modules 
from pypdf import PdfReader 

# creating a pdf reader object 
reader = PdfReader('../embed_docs/Player_s Handbook.pdf') 
pages = reader.pages
page = pages[42]
print(page.extract_text(extraction_mode="layout"))

In [ ]:
import ollama
from pprint import pprint
from typing import List 

def summarize_from_pages(pages) -> List:
    task = "Summarize this in one to three sentences."
    response = []
    for page in pages:
        page_text = page.extract_text(extraction_mode="layout")
        task_response = ollama.generate(
            model='mistral:7b',
            prompt=f"Using the following context: {page_text}, perform this task: {task}"
        )
        response.append(task_response)
    return response

task_response = summarize_from_pages(pages)


In [ ]:

pprint(task_response[42])

In [ ]:
for task in task_response:
    pprint(task['response'])


Let's build a pipeline

In [15]:
pip install text_chunker tqdm numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pypdf import PdfReader 
from tqdm import tqdm

file = '../embed_docs/Player_s Handbook.pdf'

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PdfReader(file)
        text = " ".join(page.extract_text() for page in pdf.pages)
    return text

# Extract text from the PDF and split it into sentences
text = extract_text_from_pdf(file)

In [16]:
import numpy as np
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.max_length = 30000000


def process(text):
    doc = nlp(text)
    sents = list(doc.sents)
    vecs = np.stack([sent.vector / sent.vector_norm for sent in sents])

    return sents, vecs

def cluster_text(sents, vecs, threshold):
    clusters = [[0]]
    for i in range(1, len(sents)):
        if np.dot(vecs[i], vecs[i-1]) < threshold:
            clusters.append([])
        clusters[-1].append(i)
    
    return clusters

def clean_text(text):
    # Add your text cleaning process here
    return text

# Initialize the clusters lengths list and final texts list
clusters_lens = []
final_texts = []

# Process the chunk
threshold = 0.3
sents, vecs = process(text)

# Cluster the sentences
clusters = cluster_text(sents, vecs, threshold)

for cluster in clusters:
    cluster_txt = clean_text(' '.join([sents[i].text for i in cluster]))
    cluster_len = len(cluster_txt)
    
    # Check if the cluster is too short
    if cluster_len < 60:
        continue
    
    # Check if the cluster is too long
    elif cluster_len > 3000:
        threshold = 0.6
        sents_div, vecs_div = process(cluster_txt)
        reclusters = cluster_text(sents_div, vecs_div, threshold)
        
        for subcluster in reclusters:
            div_txt = clean_text(' '.join([sents_div[i].text for i in subcluster]))
            div_len = len(div_txt)
            
            if div_len < 60 or div_len > 3000:
                continue
            
            clusters_lens.append(div_len)
            final_texts.append(div_txt)
            
    else:
        clusters_lens.append(cluster_len)
        final_texts.append(cluster_txt)

In [22]:
import ollama
from pprint import pprint
from typing import List 

def summarize_from_chunks(chunks) -> List:
    task = "Summarize this in one to three sentences."
    response = []
    for chunk in enumerate(chunks):
        task_response = ollama.generate(
            model='mistral:7b',
            prompt=f"Using the following context: {chunk}, perform this task: {task}"
        )
        response.append(task_response)
    return response

task_response = summarize_from_chunks(final_texts)


In [32]:
import ollama
from pprint import pprint
from typing import List 
from tqdm import tqdm

def chunk_and_summarize(chunks) -> List:
    task = "Summarize this in one to three sentences."
    summaries_and_chunks = []
    for chunk in tqdm(enumerate(chunks)):
        summary = ollama.generate(
            model='mistral:7b',
            prompt=f"Using the following context: {chunk}, perform this task: {task}"
        )
        summary_response = summary['response']
        summaries_and_chunks.append({'summary': summary_response, 'chunk': chunk})
    return summaries_and_chunks

s_and_c = chunk_and_summarize(final_texts)

3988it [51:13,  1.30it/s]


In [ ]:
pprint(s_and_c[350])
